# Predicting Satellite Image Lan Use Classes for Urban Development

Overview: using pre-labeled satellite image data from the EuroSAT database (https://www.tensorflow.org/datasets/catalog/eurosat):

Business Case: Urban planning consultants need to filter data for a given city by it's land use. Run an image classification model on pre-trained data, and use domain knowledge to apply the model to unclassified satellite imagery.


1. Create a NN model that can accurately predict land cover using the classified dataset
1. determine best model architecture and tune hyper-parameters to acheive best accuracy scores
1. Predict type of land use on future unseen data


In [ ]:
# Importing few libraries
import os
import shutil
import random
from tqdm import tqdm

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay, recall_score, f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

import PIL
from PIL import Image

#Neural network packages
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D # creates densely connected layer object
from tensorflow.keras.layers import Dropout, BatchNormalization, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

for reference: tutorial followed on loading image data here: https://www.tensorflow.org/tutorials/images/classification

## Reviewing the Images Dataset

In [ ]:
batch_size = 32
img_height = 64
img_width = 64
DATASET = "2750"
LABELS = os.listdir(DATASET)
print(LABELS)

In [ ]:
# plot class distributions of whole dataset
counts = {}

for l in LABELS:
    counts[l] = len(os.listdir(os.path.join(DATASET, l)))
    
plt.figure(figsize=(12, 6))

plt.bar(range(len(counts)), list(counts.values()))
plt.xticks(range(len(counts)), list(counts.keys()), rotation=40)
plt.xlabel('class label')
plt.ylabel('class size')
plt.title('EUROSAT Class Distribution');

In [ ]:
#review a few samples of the training/validation dataset
img_paths = [os.path.join(DATASET, l, l+'_1.jpg') for l in LABELS]

img_paths = img_paths + [os.path.join(DATASET, l, l+'_2.jpg') for l in LABELS]

def plot_sat_imgs(paths):
    plt.figure(figsize=(3, 15))
    for i in range(20):
        plt.subplot(10, 2, i+1, xticks=[], yticks=[])
        img = PIL.Image.open(paths[i], 'r')
        plt.imshow(np.asarray(img))
        #plt.title(paths[i].split('/')[-2])

plot_sat_imgs(img_paths)

In [ ]:
#reviewing the path for the data for future pulling
img_paths[0]

In [ ]:
#reviewing the called datapoint
img1 = Image.open(img_paths[0])
img1

In [ ]:
print(f'image size: {img1.size}')
print(f'first pixel in RGB values: {img1.getpixel((0, 0))}')

In [ ]:
#confirm shape of the data
input1 = np.array(img1)
flattened_input1 = np.ravel(input1)/255 #normalized array
input1.shape

## Extracting the Data to feed into Machine Learning Model

In [ ]:
# Define parameters for loading images from the directory
target_size = (64, 64)  # Resize images to a consistent size, if not already done so
batch_size = 32  # Number of images to load at each iteration
class_mode = 'categorical'  # Use 'binary' for binary classification, 'categorical' for multi-class classification

# Create an ImageDataGenerator instance
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to be between 0 and 1
    validation_split=0.2,  #split data into training and validation sets
    rotation_range=360 #randomly rotate the images across all possible values (0-360) - will help with overfit models
    )

# Load training data
train_generator = datagen.flow_from_directory(
    DATASET,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    subset='training',  # Specify 'training' to load the training set
    seed=53 #setting seed for re-producability
)

validation_generator = datagen.flow_from_directory(
    DATASET,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    subset='validation',  # Specify 'validation' to load the validation set
    seed=53, #setting seed for re-producability
    shuffle = False
)

#create an array of the classes for future use
true_classes = []
for i in range(len(validation_generator)):
    images, labels = validation_generator[i]
    batch_labels = np.argmax(labels, axis=1)
    true_classes.append(batch_labels)

flat_list = [item for sublist in true_classes for item in sublist]
true_classes = flat_list

In [ ]:
#review the classes
class_indices = train_generator.class_indices
class_names = {v: k for k, v in class_indices.items()}
class_names

Given time restraints, the entire dataset can take up to 4 hours to run through a model; therefore, will create a "sample" dataset that is a fraction of the entire train/validation dataset, determine the best model, then fit that model to the entire dataset.

In [ ]:
# Create an ImageDataGenerator instance for a sample-sized dataset - for faster-running base models
sample_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to be between 0 and 1
    validation_split=0.2, #split data into training and validation sets
    rotation_range=360
    )

target_size = (64, 64)  # Resize images to a consistent size
batch_size = 20  # Number of images to load at each iteration
class_mode = 'categorical'  # Use 'binary' for binary classification, 'categorical' for multi-class classification

# Load sample data into train
sample_train_generator = sample_datagen.flow_from_directory(
    'sample_img',
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    subset='training',
    seed=53
)

#load sample data into validation
sample_val_generator = sample_datagen.flow_from_directory(
    'sample_img',
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    subset='validation',
    seed=53,
    shuffle = False
)

#create an array of the classes for the validation data
true_classes_sample = []
for i in range(len(sample_val_generator)):
    images, labels = sample_val_generator[i]
    batch_labels = np.argmax(labels, axis=1)
    true_classes_sample.append(batch_labels)

flat_list_sample = [item for sublist in true_classes_sample for item in sublist]
true_classes_sample = flat_list_sample

### Defining the baseline model:

In [ ]:
#setting up early stopping, as many of the models converge fairly quickly
early_stopping = EarlyStopping(monitor='accuracy',  
                               patience=20,          # number of epochs with no improvement after which training will be stopped
                               verbose=1,           # logs messages
                               restore_best_weights=True)  # restore model weights from the epoch with the best value of the monitored quantity


In [ ]:
#trying a simple flat-model, for reference
model0 = Sequential()

#adding in a flatten activation
model0.add(Flatten(input_shape=(64, 64, 3)))

model0.add(Dense(64, activation='relu', kernel_regularizer = l2(3e-3)))
model0.add(Dense(64, activation='relu', kernel_regularizer = l2(3e-3)))
model0.add(Dense(10, activation='softmax'))

model0.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])

In [ ]:
model0.summary()

In [ ]:
sample_cnn0 = model0.fit(sample_train_generator, validation_data=sample_val_generator, 
                         epochs=200, batch_size=32, verbose = 2, callbacks=[early_stopping])

In [ ]:
#review the accuracy metrics by epoch
sample_cnn_history0 = pd.DataFrame(sample_cnn0.history)
sample_cnn_history0.index.name = 'epochs'

col_list = ['accuracy', 'val_accuracy']
sample_cnn_history0[col_list].plot()
plt.ylabel('accuracy')
plt.title('Training accuracy history')
plt.show()

### Flat model understandably isn't very accurate, moving on to a baseline CNN model:

In [ ]:
model_sample = Sequential()
# define 3x3 filter window sizes. Create 32 filters.
# COv2D input shape =(image_height, image_width, color_channels) for each image
model_sample.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(64, 64, 3)))
# max pool in 2x2 window
model_sample.add(MaxPooling2D(pool_size=(3, 3)))

# transition to dense fully-connected part of network
model_sample.add(Flatten())
model_sample.add(Dense(64, activation='relu'))
model_sample.add(Dense(10, activation='softmax'))

In [ ]:
model_sample.summary()

In [ ]:
model_sample.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])

In [ ]:
sample_cnn = model_sample.fit(sample_train_generator, validation_data=sample_val_generator, 
                              epochs=200, batch_size=32, verbose = 2, callbacks=[early_stopping])

In [ ]:
sample_cnn_history = pd.DataFrame(sample_cnn.history)
sample_cnn_history.index.name = 'epochs'

col_list = ['accuracy', 'val_accuracy']
sample_cnn_history[col_list].plot()
plt.ylabel('Accuracy')
plt.title('Training accuracy history - Model 1 sample')
plt.show()

### Model runs much better than the flat Neural Network - try new regularized model on sample data before feeding in entire dataset

In theory, regularization should help reduce the overfitting going on by avoiding the weights from exploding.

In [ ]:
model2 = Sequential()
# define 3x3 filter window sizes. Create 32 filters.
# COv2D input shape =(image_height, image_width, color_channels) for each image
model2.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(64, 64, 3),
                        kernel_regularizer = l2(3e-3) ))
# max pool in 2x2 window
model2.add(MaxPooling2D(pool_size=(4, 4)))

model2.add(Conv2D(filters=32,
                        kernel_size=(4, 4),
                        activation='relu',
                        kernel_regularizer = l2(3e-3) ))

# transition to dense fully-connected part of network
model2.add(Flatten())
model2.add(Dense(64, activation='relu', kernel_regularizer = l2(3e-3)))
model2.add(Dense(10, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
sample_cnn2 = model2.fit(sample_train_generator, validation_data=sample_val_generator, 
                         epochs=100, batch_size=32, verbose = 2, callbacks=[early_stopping])

In [ ]:
sample_cnn_history2 = pd.DataFrame(sample_cnn2.history)
sample_cnn_history2.index.name = 'epochs'
best_epoch = sample_cnn2.epoch[-1]
print(best_epoch)
col_list = ['accuracy', 'val_accuracy']
sample_cnn_history2[col_list].plot()
plt.ylabel('accuracy')
plt.title('Training accuracy history - model 2')
plt.show()

### This runs slightly better than the original base model; add Dropout filters:

In [ ]:
#addingdropout layers
model3 = Sequential()

# define 3x3 filter window sizes. Create 32 filters.
# COv2D input shape =(image_height, image_width, color_channels) for each image
model3.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(64, 64, 3),
                        kernel_regularizer = l2(3e-3) ))
# max pool in 2x2 window
model3.add(MaxPooling2D(pool_size=(4, 4)))

model3.add(Dropout(0.25))
model3.add(Conv2D(filters=32,
                        kernel_size=(4, 4),
                        activation='relu',
                        kernel_regularizer = l2(3e-3) ))
model3.add(Dropout(0.25))
# transition to dense fully-connected part of network
model3.add(Flatten())
model3.add(Dense(64, activation='relu', kernel_regularizer = l2(3e-3)))
model3.add(Dense(10, activation='softmax'))

model3.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])

In [ ]:
model3.summary()

In [ ]:
sample_cnn3 = model3.fit(sample_train_generator, validation_data=sample_val_generator, 
                         epochs=100, batch_size=32, verbose = 2, callbacks=[early_stopping])

In [ ]:
sample_cnn_history3 = pd.DataFrame(sample_cnn3.history)
sample_cnn_history3.index.name = 'epochs'
col_list = ['accuracy', 'val_accuracy']
sample_cnn_history3[col_list].plot()
plt.ylabel('accuracy')
plt.title('Training accuracy history - model 3')
plt.show()

### Model #4 - adding more layers

In [ ]:
#addingdropout layers
model4 = Sequential()

# define 3x3 filter window sizes. Create 32 filters.
# COv2D input shape =(image_height, image_width, color_channels) for each image
model4.add(Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(64, 64, 3),
                        kernel_regularizer = l2(3e-3) ))
# max pool in 2x2 window
model4.add(MaxPooling2D(pool_size=(4, 4)))

model4.add(Dropout(0.25))
model4.add(Conv2D(filters=32,
                        kernel_size=(4, 4),
                        activation='relu',
                        kernel_regularizer = l2(3e-3) ))
model4.add(Conv2D(filters=16,
                        kernel_size=(4, 4),
                        activation='relu',
                        kernel_regularizer = l2(3e-3) ))
model4.add(Dropout(0.25))
model4.add(Conv2D(filters=16,
                        kernel_size=(2, 2),
                        activation='relu',
                        kernel_regularizer = l2(3e-3) ))

# transition to dense fully-connected part of network
model4.add(Flatten())
model4.add(Dense(64, activation='relu', kernel_regularizer = l2(3e-3)))
model4.add(Dense(32, activation='relu'))
model4.add(Dense(10, activation='softmax'))

model4.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])

In [ ]:
model4.summary()

In [ ]:
sample_cnn4 = model4.fit(sample_train_generator, validation_data=sample_val_generator, 
                         epochs=100, batch_size=32, verbose = 2, callbacks=[early_stopping])

In [ ]:
sample_cnn_history4 = pd.DataFrame(sample_cnn4.history)
sample_cnn_history4.index.name = 'epochs'
col_list = ['accuracy', 'val_accuracy']
sample_cnn_history4[col_list].plot()
plt.ylabel('accuracy')
plt.title('Training accuracy history - model 4')
plt.show()

### Review the accuracy graphs for each model, in order to determine which model to run Cross-validation on:

In [ ]:
#plot all the accuracy results on one cell

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))

col_list = ['accuracy', 'val_accuracy']

sns.lineplot(data=sample_cnn_history[col_list], ax=axs[0,0])
sns.lineplot(data=sample_cnn_history2[col_list], ax=axs[0,1])
sns.lineplot(data=sample_cnn_history3[col_list], ax=axs[1,0])
sns.lineplot(data=sample_cnn_history4[col_list], ax=axs[1,1])

axs[0,0].set_title('Model 1 - baseline')
axs[0,1].set_title('Model 2 - regularized')
axs[1,0].set_title('Model 3 - regularized with dropout')
axs[1,1].set_title("Model 4 - regularized, dropouts, add'l layers")

plt.show()

#### Given the above performances, will use the model with the best train/validation accuracy scores, and see if training on the entire dataset results in higher accuracy. 

In [ ]:
print('model 1:'+str(sample_cnn_history[col_list].max()))
print('model 2:'+str(sample_cnn_history2[col_list].max()))
print('model 3:'+str(sample_cnn_history3[col_list].max()))
print('model 4:'+str(sample_cnn_history4[col_list].max()))

In [ ]:
#(saving all models!)
model0.save('model0.h5')
model_sample.save('model1.h5')
model2.save('model2.h5')
model3.save('model3.h5')
model4.save('model4.h5')

## Tuned model ran on full labeled dataset:

In [ ]:
#note - the full model fit will take ~3 hours; loaded the sample-trained model, fitting, and re-saving the model
loaded_model = load_model('model3.h5')

In [ ]:
full_model_cnn = loaded_model.fit(train_generator, 
                              epochs=200, batch_size=32, verbose = 2, callbacks=[early_stopping])

In [ ]:
loaded_model.save('loaded_model.h5')

In [ ]:
cnn_history1 = pd.DataFrame(full_model_cnn.history)
cnn_history1.index.name = 'epochs'

col_list = ['accuracy']
cnn_history1[col_list].plot()
plt.ylabel('Accuracy')
plt.title('Training accuracy history - Full Model')
plt.show()

In [ ]:
#repeated the loading, as the notebook was shut down after training
loaded_model = load_model('loaded_model.h5')

In [ ]:
#predicting on the unseen validation data
predictions_full = loaded_model.predict(validation_generator)
#class labels
predicted_labels_full = np.argmax(predictions_full, axis=1)

In [ ]:
#review the confusion matrix for the full model
confusion_full = confusion_matrix(true_classes, predicted_labels_full)
disp4 = ConfusionMatrixDisplay(confusion_matrix=confusion_full)
disp4.plot()
legend_text = '\n'.join([f'{key}: {value}' for key, value in class_names.items()])
plt.text(15, 6, legend_text)
plt.show()

In [ ]:
accuracy = accuracy_score(true_classes, predicted_labels_full)
print("Accuracy:", accuracy)
recall = recall_score(true_classes, predicted_labels_full, average='weighted')
print("Recall:", recall)
f1 = f1_score(true_classes, predicted_labels_full, average='weighted')
print("F1 Score:", f1)

Using the full dataset, we received slightly higher accuracy scores at 82%. I may use this model in the future and do some hyper-parameter tuning, when time and resources are available. some notes on future steps:

1. ReLu appears to be the best activation methodology for neural network layers, however it would be interesting to try other available methodologies (tanh, linear, sigmoid etc.)
2. the filters were largely set at 32, however it would be interesting to evaluate different metrics here
3. ridge regularizers were exclusively used, yet I would like to evaluate lasso, as well as different strengths

# FINAL MODEL RUN ON UNSEEN DATASET

Using my domain knowledge (born and raised in this area!), I would like to take satellite imagery from the Springfield, MA metropolitan area and see how well the model can classify sections of the image. (taken from Google Earth, 25 October 2023).

Steps are as follows:

1. load in the image
1. create "chunks" of the image - given that the model was trained on 64x64 pixel images, i will "chunk" the satellite image into these proportions. NOTE: for future improvement, I will rescale based on any given image size. However, I ensured the satellite image I captured had a width & height that is divisible by 64, to ensure the chunks are evenly spaced for future plotting
1. save the chunks, feed into an array that will fit into the trained full model
1. predict the classifications on each of the chunks
1. Plot the chunk classifications back onto the original Satellite image
- this ended up becoming more manual than I expected - notes below

In [ ]:
#import test image of Springfield, MA metro area subset
test_image = Image.open('SPFLD_test.jpg')
test_image


In [ ]:
#check the size
test_image.size

### Saving the breakout of the test image in folder `test_img` - only need to run this code once!

In [ ]:
# Define the chunk size (square shape, number equaling each pixel width/height)
chunk_size = 64

# Get the dimensions of the original image
width, height = test_image.size

# Iterate through the original image in 64x64 chunks
for i in range(0, height, chunk_size):
    for j in range(0, width, chunk_size):
        # Define the region to crop
        left = j
        upper = i
        right = j + chunk_size
        lower = i + chunk_size

        # Crop the chunk from the original image
        chunk = test_image.crop((left, upper, right, lower))

        # Save the chunk as a new image - the coordinates in the naming convention are crucial for future plotting!
        chunk.save(f"test_img/chunk_{i}_{j}.jpg")

In [ ]:
#chunk_files = os.listdir("test_img")

# Initialize an empty array to store the chunks
chunks_data = []
chunk_file = []
width, height = test_image.size
chunk_size = 64

for i in range(0, height, chunk_size):
    for j in range(0, width, chunk_size):
        # Define the region to crop
        left = j
        upper = i
        right = j + chunk_size
        lower = i + chunk_size
    # Load each chunk and preprocess it - this is where the naming convention ensures it is loaded in the correct order!
        chunk_file.append(f"test_img/chunk_{i}_{j}.jpg")
        chunk = Image.open(os.path.join(f"test_img/chunk_{i}_{j}.jpg")).resize((64, 64))
        chunk_array = np.array(chunk) / 255.0  # Normalize the pixel values between 0 and 1
        chunks_data.append(chunk_array)

# Convert the list of chunks into a NumPy array
test_data = np.array(chunks_data)

In [ ]:
#ensure the shape is correct, especially in dimensions 2-4
test_data.shape

In [ ]:
#check that chunk file #1 is chunk_0_64
chunk_file[]

In [ ]:
#un-hash the below to upload the saved array - for future work
# Access the test_data array from the loaded file
#test_data = loaded_data["test_data"]

In [ ]:
#classify each image chunk using the loaded model from above
test_preds = loaded_model.predict(test_data)
predicted_classes_test = np.argmax(test_preds, axis=1)

In [ ]:
#ensure the shape is still accurate
predicted_classes_test.shape

In [ ]:
#brief EDA on the predictions
# Get the unique values and their counts
unique_values, counts = np.unique(predicted_classes_test, return_counts=True)

# Print the unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"Class: {value}, Count: {count}")

In [ ]:
#reminder of the class names for each value 0-9:
list(class_names.values())

In [ ]:
#EDA on one given class, Industrial:
display = 4    
display_vals = [index for index, value in enumerate(predicted_classes_test) if value == display][:15]
display_vals

In [ ]:
#show images that were labeled as class defined under variable "display" above - in this case, Industrial:
display = 4    
display_vals = [index for index, value in enumerate(predicted_classes_test) if value == display][:15]
    
def plot_test_imgs(paths):
    plt.figure(figsize=(15, 8))
    for i in range(15):
        plt.subplot(3, 5, i+1, xticks=[], yticks=[])
        img = Image.open(os.path.join(chunk_file[display_vals[i]]))
        plt.imshow(np.asarray(img))

plot_test_imgs(display_vals)

#### Moving on to plotting the chunks back onto the original shape with their class.

Note: a bit manual, in the future I plan to automate the below sections for ease of visualization.

1. Create a NumPy array ot zeroes that matches the size of the original image in pixels
1. iterate through the chunks (IN THE SAME ORDER THAT THE CHUNKS ARE STORED IN THE INDEX) and assign the predicted class to the pixels that the chunk correlates to in the original image
1. use .imshow (https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html) to plot each class over the original image. for visibility, I played around with the "alpha" (transparency) and landed largely on 0.5.

In [ ]:
# Initialize an empty array to store the predicted classes mapped to the original image shape
mapped_predictions = np.zeros((4224, 8192))
chunk_size = 64

# Iterate through the chunks and map predictions back to the original image
index = 0
for i in range(0, 4224, chunk_size):
    for j in range(0, 8192, chunk_size):
        # Get the predicted class for the current chunk
        predicted_class = predicted_classes_test[index]

        # Map the predicted class to the corresponding region in the original image
        mapped_predictions[i:i+chunk_size, j:j+chunk_size] = predicted_class
        index += 1

In [ ]:
#once again, confirm the size. NOTE: for array, width and height are flipped. this makes sense, 
# since we are looking at an array in the notation of (rows, columns) instead of (width, height)
test_image.size

In [ ]:
#confirming inverse of the image size is the numpy array shape
mapped_predictions.shape


In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(10, 10))

#clear xticks - we just want to see the image for now
ax.set_xticks([])
ax.set_yticks([])
plotindex=0

#loop through each class and make a separate plot for each
for i in range (5):
    for j in range(2):
        if plotindex < len(class_names):
            # Plot the original image
            axs[i,j].imshow(test_image)
            #plot the predicted class by index
            mask = (mapped_predictions == plotindex)
            axs[i,j].imshow(mask, alpha=0.5)
            axs[i,j].set_title(list(class_names.values())[plotindex])
            axs[i,j].set_xticks([])
            axs[i,j].set_yticks([])
        else:
            axs[i,j].axis('off') 
        plotindex += 1

plt.subplots_adjust(wspace=0.01, hspace=0.01)
plt.tight_layout()
plt.show()

In [ ]:
# Reviewing a few classes at larger scale - again, starting with industrial
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xticks([])
ax.set_yticks([])
# Plot the original image
ax.imshow(test_image)
#plot the class - 'Industrial'
mask = (mapped_predictions == 4)
ax.imshow(mask, alpha=0.5)#, cmap='coolwarm')

In [ ]:
# doing the same for "Herbaceous vegetation"
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xticks([])
ax.set_yticks([])
# Plot the original image
ax.imshow(test_image)
#plot the class - 'LakeSea'
mask = (mapped_predictions == 2)
ax.imshow(mask, alpha=0.5)#, cmap='coolwarm')

In [ ]:
#attempt looking at all the classes on the same map by color gradient
# Define a colormap with a unique color for each class
num_classes = len(np.unique(mapped_predictions))
colors = plt.cm.get_cmap('tab20', num_classes) 

fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xticks([])
ax.set_yticks([])

# Plot the original image
ax.imshow(test_image)
ax.imshow(mapped_predictions, cmap='viridis', interpolation='nearest', alpha=0.4)

ax.legend()
plt.show()

Note - I wasn't sure where this would go, but it doesn't appear to be giving much value when combining all. Keeping for future use, as this may prove useful in other test images.

One last visualization for curiosity - looking at the graph of each class mapped out, it appears that the water features are classified as river, Sea/Lake, and even forest (note: it appears this is due to the "forest" training data having a very dark green/blue hue). How would it look if all these classifications were wrapped into one image/class?

In [ ]:
#just for fun
# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xticks([])
ax.set_yticks([])
# Plot the original image
ax.imshow(test_image)
#plot the class - 'LakeSea', 'forest', or 'river'
mask = np.isin(mapped_predictions, [1, 8, 9])
ax.imshow(mask, alpha=0.5)#, cmap='coolwarm')
ax.set_title('combined forest, river, sea/lake classifications')

It's actually much more accurate! There is potential here for "class refinement" for future models...

In [ ]:
#One last "just for fun" - see if it can predict where the people live.
# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xticks([])
ax.set_yticks([])
# Plot the original image
ax.imshow(test_image)
#plot the class - 'LakeSea', 'forest', or 'river'
mask = np.isin(mapped_predictions, [7])
ax.imshow(mask, alpha=0.5)#, cmap='coolwarm')
ax.set_title('Residential')

# Conclusion

Overall, the model performed well at 82% accuracy. As mentioned throughout, there are multiple paths to go in order to improve the model, though arguably the best factor would be incremental data on a like-for-like satellite imagery. 